In [25]:
require 'numo/narray'
require 'numo/gsl'
require 'numo/gnuplot'
require 'ostruct'
require 'daru'
require 'fileutils'
require 'pry'
require 'spreadsheet'

false

In [26]:
IRuby::Kernel.instance.switch_backend!(:pry)

true

In [27]:
def plotify(filename)
  Numo.noteplot do |note|
    set terminal: 'pdfcairo font "Gill Sans,9" size 29cm, 21cm linewidth 2 rounded fontscale 0.75'
    set output: filename
    
    set style: 'line 80 lt rgb "#808080"'
    set style: 'line 81 lt 0'
    
    set lmargin: 15
    set rmargin: 15
    set tmargin: 5
    set bmargin: 5

    # Remove border on top and right.  These
    # borders are useless and make it harder
    # to see plotted lines near the border.
    # Also, put it in grey; no need for so much emphasis on a border.
    set border: '3 back linestyle 80'
    set grid: 'back linestyle 81'
    set xtics: 'nomirror'
    set ytics: 'nomirror'
    
    set key: 'bottom right'
    
    # Line styles: try to pick pleasing colors, rather
    # than strictly primary colors or hard-to-see colors
    # like gnuplot's default yellow.  Make the lines thick
    # so they're easy to see in small plots in papers.
    set style: 'line 1 lt rgb "#A00000" lw 1 pt 1'
    set style: 'line 2 lt rgb "#00A000" lw 1 pt 6'
    set style: 'line 3 lt rgb "#5060D0" lw 1 pt 2'
    set style: 'line 4 lt rgb "#F25900" lw 1 pt 9'
    
    yield note
  end
end

:plotify

In [134]:
df = Daru::DataFrame.from_csv('plot.csv')

,T,P,ErrT,ErrP
0,900,9.875,10,0.07830000000000001
1,1000,14.650739999999999,10,0.09107
2,1100,23.374000000000002,10,0.1096
3,1200,33.374,10,0.1272
4,1300,46.4,10,0.1464
5,1400,58.32000000000001,10,0.162
6,1500,69.34499999999998,10,0.17529999999999998
7,1600,84.993,10,0.1921
8,1700,102.18,10,0.20900000000000002
9,1800,123.61999999999999,10,0.2283


In [135]:
T, P = df['T'], df['P']
df['lnT'], df['lnP'] = df['T'], df['P']
df['lnT'] = df['T'].map { |e| Math.log(e) }
df['lnP'] = df['P'].map { |e| Math.log(e) }
T_err, P_err = df['ErrT'], df['ErrP']
df['Err lnT'] = T_err * df['T']**-1
df['Err lnP'] = P_err * df['P']**-1
a = df['lnP']

(pry):1: warning: already initialized constant T
(pry):1: warning: previous definition of T was here
(pry):1: warning: already initialized constant P
(pry):1: warning: previous definition of P was here
(pry):5: warning: already initialized constant T_err
(pry):5: warning: previous definition of T_err was here
(pry):5: warning: already initialized constant P_err
(pry):5: warning: previous definition of P_err was here


,lnP
0,2.2900063107871858
1,2.684490846132783
2,3.1516242935109653
3,3.507777153724711
4,3.8372994592322094
5,4.065945087700403
6,4.239094046108298
7,4.4425689001579505
8,4.626735963902652
9,4.817212344231127


In [136]:
x, y = df['lnT'], df['lnP']
x_err, y_err = df['Err lnT'], df['Err lnP']

n_samples = 1000

nx = Numo::DFloat[x.to_a]
ny = Numo::DFloat[y.to_a]

best_fit = Numo::GSL::Fit.linear(nx, ny)
xrange = Numo::DFloat.linspace(nx[0], nx[-1], n_samples)
best_fit_y, best_fit_err = *Numo::GSL::Fit.linear_est(xrange, best_fit)
c0 = best_fit.c0[0]
c1 = best_fit.c1[0]
c0_err = Math.sqrt best_fit.cov00[0]
c1_err = Math.sqrt best_fit.cov11[0]

puts ("c0 = %1.3e +- %1.3e" % [c0, c0_err])
puts ("c1 = %1.3e +- %1.3e" % [c1, c1_err])

c0 = -2.198e+01 +- 7.599e-01
c1 = 3.583e+00 +- 1.052e-01


In [138]:
plotify 'plot.pdf' do |note|
  note.set xlabel: 'ln(T)'
  note.set ylabel: 'ln(P)'
  note.set title: 'lnP = f(ln(T))'
  note.set format: { y:'%2.0e' }
  note.set style: 'fill transparent solid 0.5' # partial transparency
  note.plot [x, y, x_err, y_err, {ls:1, with: 'xyerrorbars', title: 'Experimental data: ln(P)'}],
            [xrange, best_fit_y, {ls:3, with: 'lines', title: 'Best fit: c0 + c1 * ln(T)'}],
            [xrange, best_fit_y + best_fit_err, best_fit_y - best_fit_err,
                {ls:3, with: 'filledcurves', title: 'Best fit error margin'}]
end